In [1]:
import torch
from schnetpack.datasets import QM9
import schnetpack as spk
import os
from my_config import config_args
from Model.HGDM import HyperbolicAE,HyperbolicDiffusion
import optimizers
import numpy as np
import logging
import time
from tqdm import tqdm

# 获得训练数据

In [2]:
qm9data = QM9('./data/qm9.db', download=True,load_only=[QM9.U0])
qm9split = './data/qm9split'
print(len(qm9data))

133885


In [3]:
train, val, test = spk.train_test_split(
        data=qm9data,
        num_train=30000,
        num_val=10000,
        split_file=os.path.join(qm9split, "split30000-10000.npz"),
    )
print(len(train),len(val),len(test))

30000 10000 93885


In [4]:
train_loader = spk.AtomsLoader(train, batch_size=256, shuffle=False)
val_loader = spk.AtomsLoader(val, batch_size=256)

In [5]:

import json
class obj(object):
    def __init__(self, dict_):
        self.__dict__.update(dict_)
args = json.loads(json.dumps(config_args), object_hook=obj)

model = HyperbolicAE(args)

optimizer = getattr(optimizers, args.optimizer)(params=model.parameters(), lr=args.lr,
                                                    weight_decay=args.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=args.lr_reduce_freq,
    gamma=float(args.gamma)
)
tot_params = sum([np.prod(p.size()) for p in model.parameters()])
logging.info(f"Total number of parameters: {tot_params}")
device = torch.device('cuda')
# Train model
t_total = time.time()

model = model.to(device)


In [6]:
for epoch in range(args.epochs):
    model.train()
    loss_sum,n,t = 0,0,0.0
    counter = 0
    for input in tqdm(train_loader):
        # counter+=1
        # if counter>10:
        #     sys.exit(0)
        for key in input:
            input[key] = input[key].to(torch.device('cuda'))
        t = time.time()
        model.train()
        optimizer.zero_grad()
        loss = model(input)
        # print('y_hat',y_hat[:2])
        # print('input[QM9.U0]',input[QM9.U0][:2])
        # loss = loss_fun(y_hat,input[QM9.U0])
        # curvatures = list(model.get_submodule('curvatures'))
        # print(curvatures)

        # print(loss)
        loss.backward()
        loss_sum += loss
        n += 1
        if args.grad_clip is not None:
            max_norm = float(args.grad_clip)
            all_params = list(model.parameters())
            for param in all_params:
                torch.nn.utils.clip_grad_norm_(param, max_norm)
        optimizer.step()
    lr_scheduler.step()
    if (epoch + 1) % args.log_freq == 0:
        str = " ".join(['Epoch: {:04d}'.format(epoch + 1),
                               'lr: {}'.format(lr_scheduler.get_last_lr()[0]),
                               'loss: {:.4f}'.format(loss_sum/n),
                               'time: {:.4f}s'.format(time.time() - t)
                               ])
        print(str)
        # logging.info(str)
        # curvatures = list(model.get_submodule('encoder.curvatures'))
        # print('encoder:',curvatures)
        # curvatures = list(model.get_submodule('decoder.curvatures'))
        # print('decoder:',curvatures)

    model.eval()
    with torch.no_grad():
        loss_sum,n = 0.0,0
        for input in tqdm(val_loader):
            for key in input:
                input[key] = input[key].to(torch.device('cuda'))
            t = time.time()
            model.train()
            optimizer.zero_grad()
            loss = model(input)
            n += 1
            loss_sum += loss

        print('val_loss:',loss_sum.item()/n)

"""
Epoch: 0020 lr: 0.0025 loss: 24.7880 time: 0.0977s
Epoch: 0020 lr: 0.0025 loss: 13.2554 time: 0.0969s
"""

 14%|█▎        | 16/118 [00:08<00:55,  1.83it/s]


KeyboardInterrupt: 

In [ ]:

"""
hyperbolid
val_loss: tensor(0.0009, device='cuda:0') encoder结束后投影到欧氏空间
val_loss: tensor(1.5983e-05, device='cuda:0') encoder结束后保持在双曲空间
欧氏空间：
val_loss: tensor(0.1687, device='cuda:0')

val_loss: tensor(0.2672, device='cuda:0') 坐标在双曲空间 encoder结束后保持在双曲空间
val_loss: tensor(0.0075, device='cuda:0') 坐标在双曲空间 encoder结束后投影到欧氏空间
"""

In [7]:
# torch.save(model.encoder.state_dict(), './saved_model/HNN-encoder.pt')
encoder_path = './saved_model/HNN-encoder.pt'
# torch.save(model.decoder.state_dict(), './saved_model/HNN-decoder.pt')
decoder_path = './saved_model/HNN-decoder.pt'

In [8]:
from Model import Encoders, Decoders

encoder = getattr(Encoders, args.model)(args)
encoder.load_state_dict(torch.load(encoder_path))
decoder = Decoders.model2decoder[args.model](encoder.curvatures, args)
decoder.load_state_dict(torch.load(decoder_path))

<All keys matched successfully>

In [9]:
model = HyperbolicDiffusion(args,encoder,decoder)
model = model.to(device)
model.eval()
with torch.no_grad():
    loss_sum,n = 0.0,0
    for input in tqdm(train_loader):
        for key in input:
            input[key] = input[key].to(torch.device('cuda'))
        t = time.time()
        model.train()
        optimizer.zero_grad()
        loss = model(input)
        print(loss)
        n += 1
        loss_sum += loss

    # print('loss:',loss_sum/n)

Gamma schedule:
[-5.         -4.693911   -4.388096   -4.0821896  -3.7761917  -3.4702852
 -3.1642876  -2.8584723  -2.5523832  -2.246203   -1.940388   -1.6344814
 -1.3283925  -1.0223949  -0.71630573 -0.4102168  -0.10421944  0.20186996
  0.5079589   0.81395626  1.1198626   1.426043    1.732223    2.0384035
  2.3444924   2.6504898   2.9566703   3.2629414   3.5688477   3.8751192
  4.181391    4.487571    4.7935686   5.0999317   5.4061117   5.7123823
  6.01838     6.3246517   6.630924    6.937195    7.2432833   7.549555
  7.855918    8.162098    8.4683695   8.7745495   9.081003    9.387275
  9.693546   10.        ]


  0%|          | 0/118 [00:00<?, ?it/s]


ValueError: only one element tensors can be converted to Python scalars